# Impacto de Meteorito

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("Meteorite_Landings.csv")
df.info()

In [ ]:
df.head()

### Investigar y explorar datos

In [ ]:
df2 = df.dropna(subset=["mass", "year"])  # Eliminamos valores nan de cada una de esas columnas
df2 = df2[df2["mass"] > 0]  # Filtramos por la columna mass sea mayor estricto a cero

In [ ]:
df2.info()

Exploremos también los datos visualmente.

In [ ]:
pd.plotting.scatter_matrix(df[["mass", "year", "reclat", "reclong"]], figsize=(7,7));

In [ ]:
year = df2["year"]
masses = df2["mass"]
logmass = np.log(masses)
plt.hist(logmass, bins=100);

In [ ]:
counts = year.value_counts()
plt.scatter(counts.index, counts)
plt.xlim(1980, 2020);

In [ ]:
from scipy.stats import norm, skewnorm, lognorm
ms = np.linspace(-5, 20, 100)

mean, std = logmass.mean(), np.std(logmass)
pdf_norm = norm.pdf(ms, mean, std)
plt.hist(logmass, bins=100, density=True, alpha=0.5)
plt.plot(ms, pdf_norm);

In [ ]:
p_lognorm = lognorm.fit(logmass)
pdf_lognorm = lognorm.pdf(ms, *p_lognorm)

p_skewnorm = skewnorm.fit(logmass)
pdf_skewnorm = skewnorm.pdf(ms, *p_skewnorm)

plt.hist(logmass, bins=100, density=True, alpha=0.2, label="Datos")
plt.plot(ms, pdf_norm, label="Normal")
plt.plot(ms, pdf_lognorm, label="Log norm")
plt.plot(ms, pdf_skewnorm, label="normal simetrica");
plt.legend()
plt.xlabel("Log mass")
plt.ylabel("Probabilidad");

In [ ]:
mass_of_doom = np.log((4/3) * np.pi * 500**3 * 1600 * 1000)  # Solo usando una aproximación esférica y algo de densidad promedio (formula del area de una esfera)
mass_of_doom

In [ ]:
ms2 = np.linspace(-5, 40, 200)
plt.plot(ms2, lognorm.logsf(ms2, *p_lognorm))
plt.axvline(mass_of_doom, ls="--")
plt.xlabel("log mass")
plt.ylabel("log probabilidad")
plt.title("Log probabilidad de un asteroide con una masa dada");

Así que aquí tenemos la probabilidad de que un asteroide esté por encima de cierta masa cuando golpee la Tierra. Pero para responder a la pregunta "¿Cuál es la probabilidad de que uno o más asteroides de gran masa golpeen la Tierra en 1000 años?" tenemos que tener en cuenta el componente de tiempo real. Supongamos que en los próximos 1000 años, predecimos que habrá impactos de $N$.

$$P(>=1\ {\rm altamasa}) = 1 - P(0\ {\rm altamasa}) = 1 - P(N\ {\rm no\_altamasa}) = 1 - P({\rm no\_alta masa})^N$$

Imagine una pregunta similar: Probabilidad de no obtener seises en 5 tiradas. Bueno, es 
$$\frac{5}{6}\times\frac{5}{6}\times\frac{5}{6}\times\frac{5}{6}\times\frac{5}{ 6} = \left(\frac{5}{6}\right)^5 $$
La probabilidad de obtener uno o más seises es entonces $1 - (5/6)^5$.

Entonces, para dar un número, necesitamos calcular $N$ a partir de la tasa anual, la cantidad de años y nuestra eficiencia de detección y usar eso con la probabilidad de que cualquier impacto dado no sea de gran masa.

In [ ]:
prob_small_mass = lognorm.logcdf(mass_of_doom, *p_lognorm)
frac_sky_monitored = 0.2
num_years = 1000
num_events = num_years * counts.max() / frac_sky_monitored
prob_bad = 1 - np.exp(num_events * prob_small_mass)
print(f"Prob un impacto de asteroide >1km con {num_years} años es {prob_bad * 100:.2f}%")